In [1]:
println(s"Current spark version is ${spark.version}")

Current spark version is 2.4.5


In [2]:
import org.apache.spark.sql.types.{StructType, StructField, IntegerType, LongType, StringType}

val dataSchema = new StructType()
    .add("target", IntegerType)
    .add("id", LongType)
    .add("raw_timestamp", StringType)
    .add("query_status", StringType)
    .add("author", StringType)
    .add("tweet", StringType)

val dataPath= "/home/jovyan/data/training.1600000.processed.noemoticon.csv"

val raw_sentiment = spark.read
    .format("csv")
    .option("header",false)
    .schema(dataSchema)
    .load(dataPath)
    .selectExpr("(case when target=4 then 1 else 0 end) as label","tweet")

raw_sentiment.groupBy($"label").count.show

+-----+------+
|label| count|
+-----+------+
|    1|800000|
|    0|800000|
+-----+------+



dataSchema = StructType(StructField(target,IntegerType,true), StructField(id,LongType,true), StructField(raw_timestamp,StringType,true), StructField(query_status,StringType,true), StructField(author,StringType,true), StructField(tweet,StringType,true))
dataPath = /home/jovyan/data/training.1600000.processed.noemoticon.csv
raw_sentiment = [label: int, tweet: string]


[label: int, tweet: string]

In [20]:
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.feature.{HashingTF, Tokenizer}
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.sql.Row

val tokenizer = new Tokenizer()
    .setInputCol("tweet")
    .setOutputCol("words")

val hashingTF = new HashingTF()
    .setNumFeatures(1000)
    .setInputCol(tokenizer.getOutputCol)
    .setOutputCol("features")

val lr = new RandomForestClassifier()
    .setNumTrees(10)

val pipeline = new Pipeline()
  .setStages(Array(tokenizer, hashingTF, lr))


tokenizer = tok_b88a8ea33a48
hashingTF = hashingTF_4d43a39c93b1
lr = rfc_bcd935a04045
pipeline = pipeline_eafa8d640b8a


pipeline_eafa8d640b8a

In [21]:
val model = pipeline.fit(raw_sentiment)

model = pipeline_eafa8d640b8a


pipeline_eafa8d640b8a

In [22]:
model.write.overwrite().save("/home/jovyan/models/spark-ml-model")

In [23]:
val sameModel = PipelineModel.load("/home/jovyan/models/spark-ml-model")

sameModel = pipeline_eafa8d640b8a


pipeline_eafa8d640b8a

In [24]:
val predictionsDF = sameModel.transform(raw_sentiment)

predictionsDF.show()

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|label|               tweet|               words|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|    0|@switchfoot http:...|[@switchfoot, htt...|(1000,[7,14,21,54...|[4.36068099256288...|[0.43606809925628...|       1.0|
|    0|is upset that he ...|[is, upset, that,...|(1000,[170,193,22...|[5.02122127751674...|[0.50212212775167...|       0.0|
|    0|@Kenichan I dived...|[@kenichan, i, di...|(1000,[10,36,77,1...|[5.57104513637828...|[0.55710451363782...|       0.0|
|    0|my whole body fee...|[my, whole, body,...|(1000,[82,191,296...|[5.65325532419100...|[0.56532553241910...|       0.0|
|    0|@nationwideclass ...|[@nationwideclass...|(1000,[18,96,130,...|[4.98904046693209...|[0.49890404669320...|       1.0|
|    0|@

predictionsDF = [label: int, tweet: string ... 5 more fields]


[label: int, tweet: string ... 5 more fields]

In [25]:
// predictionsDF.schema
// predictionsDF.describe()
// predictionsDF.stat
predictionsDF.printSchema()

root
 |-- label: integer (nullable = false)
 |-- tweet: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [26]:
import org.apache.spark.sql.functions._

val getProbability = udf((prediction: org.apache.spark.ml.linalg.Vector) => prediction(1))

getProbability = UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))


UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))

In [27]:
predictionsDF.select(getProbability($"probability").alias("clean_probability")).show

+-------------------+
|  clean_probability|
+-------------------+
| 0.5639319007437114|
|0.49787787224832536|
|0.44289548636217113|
|0.43467446758089984|
| 0.5010959533067908|
| 0.5115009639688723|
| 0.5149287430143249|
| 0.5512247323001278|
| 0.5046749447154798|
| 0.5055209455389142|
| 0.5115009639688723|
|0.43467446758089984|
|0.45994635712817067|
| 0.5797040126634119|
|0.49176463218840594|
| 0.5265982285238948|
| 0.5055209455389142|
| 0.5115009639688723|
|  0.533166735755633|
| 0.5448869177916822|
+-------------------+
only showing top 20 rows

